In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces = 2)
mpDraw = mp.solutions.drawing_utils

while True:
    succes, img = cap.read()
        
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = faceMesh.process(imgRGB) 
    
    if result.multi_face_landmarks:
        for faceLms in result.multi_face_landmarks:
            # Mostrar todos los puntos
            mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACEMESH_CONTOURS)
            
            # Mostrar solo el contorno
            draw_spec = mpDraw.DrawingSpec(thickness=1, circle_radius=0)  # Oculta los puntos
            mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACEMESH_CONTOURS, draw_spec, draw_spec)

        
    cv2.imshow("Image", img)
    cv2.waitKey(1)